# Computing TMA and Cohort-wise QC Metrics Notebook

This notebook requires that you run `5_rename_and_reorganize.ipynb` beforehand. After you have your reorganized data, it may be useful to investigate if there are any widespread issues for TMAs, and for cohorts.


The purpose of this notebook is to run QC checks not on a run by run basis, but on:
- FOVs across a TMA
- Batches of Tissues

There are two parts which can be done in any order, depending on which type of QC effects are of interest to you.

In [ ]:
import os
from toffy import qc_comp, qc_metrics_plots

## QC TMA Metrics

We can utilize QC Metrics to validate that are no spatial biases across your TMAs, or if they do exist, identify where they are most prevalent.

### 0. Prerequisites

In order to use the QC TMA Metrics functionality, you need to have run `5_rename_and_reorganize.ipynb` beforehand. In addition, each FOV within the cohort should be suffixed with the Row number and Column number it's associated TMA.

For example, use Notebook 5 to combine, reorganize and rename the runs to something akin to `MY_COHORT_TMA1_R1C1` and `MY_COHORT_TMA11_R10C11`.

To make use of the QC TMA Metrics, your cohort should look like something below:


```sh
my_cohort_tmas/
├── MY_COHORT_TMA1_R1C1/
├── MY_COHORT_TMA1_R1C2/
├── ...
├── MY_COHORT_TMA2_R1C2/
├── ...
├── MY_COHORT_TMA2_R7C10/
├── ...
└── MY_COHORT_TMA11_R11C10/
```

It is necessary that the Row number and Column number values are the suffixes for each FOV.

### 1. Select QC Metrics and TMAs

Specify the names of the relevant folders:
- `cohort_image_dir`: The directory where many cohorts may be stored.
- `cohort_name`: The name of the cohort containing ready-to-analyze TIFFs. 
- `cohort_path`: The path to the cohort.
- `qc_tma_metrics_dir`: The path where the QC TMA metrics should be saved.

In [ ]:
cohort_path = "D:\\Cohorts\\20220101_new_cohort"

In [ ]:
qc_tma_metrics_dir = os.path.join(cohort_path, "qc_tma_metrics")

if not os.path.exists(qc_tma_metrics_dir):
    os.makedirs(qc_tma_metrics_dir)

Set `tmas` to a list of TMAs you wish to compute QC metrics for.

In [ ]:
# Change the `tmas` variable to be a list of the tmas you want to run the QC on
tmas = ["MY_COHORT_TMA1", "MY_COHORT_TMA1", "MY_COHORT_TMA1"]

Select any combination of the following three QC metrics:
1. `"Non-zero mean intensity"`
2. `"Total intensity"`
3. `"99.9% intensity value"`

In [ ]:
qc_metrics = ["Non-zero mean intensity"]

### 2. Compute QC TMA Metrics

Initialize the QCTMA class with the path to your cohort (`cohort_path`), the path to the folder containing the QC TMA metrics (`qc_tma_metrics_dir`) and the QC Metrics of interest themselves (`qc_metrics`).

Then compute the QC metrics per FOV. FOVs which already have QC metrics files do not get theirs recomputed.

In [ ]:
qc_tmas = qc_comp.QCTMA(
    qc_metrics=qc_metrics,
    cohort_path=cohort_path,
    metrics_dir=qc_tma_metrics_dir,
)

qc_tmas.compute_qc_tma_metrics(tmas=tmas)

You may want to exclude channels depending on their impact, the `channel_exclude` variable will filter out those channels when creating the ranked QC metrics.

The following channels will *always* be excluded from the TMA Metrics ranking below:
- Au
- Fe
- Na
- Ta
- Noodle

In [ ]:
channel_exclude = ["chan_39", "chan_45"]

### 3. Compute the Ranked QC TMA Metrics

In [ ]:
qc_tmas.qc_tma_metrics_rank(tmas=tmas, channel_exclude=channel_exclude)

### 3. Plot the QC TMA Metrics

The following plot below depicts a heatmap of the TMA along with a histogram. 

The TMA QC metrics are processed first by a FOV wise ranking for each channel, then each channel is averaged for each FOV.

What we are looking for is that any particular region's average rank isn't higher than any other. An issue arises when, say all FOVs in the upper left corner of the TMA are systematically brighter than the others.


These plots get saved in a `figures` subfolder within `qc_tma_metrics_dir`.

<div align="center">
    <img src="img/nb6_ex_avg_tma_rank.png" />
</div>


In [ ]:
qc_metrics_plots.qc_tmas_metrics_plot(qc_tmas=qc_tmas, tmas=tmas, save_figure=True, dpi=300)

## QC Longitudinal Control Metrics

The second half of this notebook is dedicated to looking at QC metrics for a particular control sample across many runs, these are called *longitudinal control* metrics.

### 0. Prerequisites

In order to make use of the QC Longitudinal Control Metrics, you need to have run `5_rename_and_reorganize.ipynb` beforehand. There is no set naming convention here for each FOV.

To make use of the LC Metrics, your cohort should consist of one control sample across several runs

```sh
my_control_sample_runs/
├── MY_CONTROL_SAMPLE_RUN1/
├── MY_CONTROL_SAMPLE_RUN2/
├── ...
├── MY_CONTROL_SAMPLE_RUN5/
├── MY_CONTROL_SAMPLE_RUN6/
└── MY_CONTROL_SAMPLE_RUN7/
```

Longitudinal Control Metrics can be computed for control sample FOVs across different cohorts. For a given control sample, we will be able to analyze it's run-to-run variance. 

### 1. Select QC metrics, FOVs, Paths, and the Batch Name

Specify the names of the relevant folders:
- `control_path`: The path to the cohort containing ready-to-analyze FOVs.
- `qc_tma_metrics_dir`: The path where the QC TMA metrics should be saved.

In [ ]:
control_path = "D:\\Cohorts\\20220101_new_cohort_controls"

In [ ]:
qc_control_effect_metrics_dir = os.path.join(cohort_path, "qc_longitudinal_control")

if not os.path.exists(qc_control_effect_metrics_dir):
    os.makedirs(qc_control_effect_metrics_dir)

Select any combination of the following three QC metrics:
1. `"Non-zero mean intensity"`
2. `"Total intensity"`
3. `"99.9% intensity value"`

In [ ]:
qc_metrics = ["Non-zero mean intensity"]

Set `control_sample_name` to an identifiable name for you to refer to.

In [ ]:
control_sample_name = "MY_CONTROL_SAMPLE"

Set `fovs` to a list of FOVs you wish to compute QC Metrics for.

In [ ]:
fovs = ["MY_CONTROL_SAMPLE_RUN1", "MY_CONTROL_SAMPLE_RUN2", "MY_CONTROL_SAMPLE_RUN3"]

You may want to include, and exclude various channels depending on their impact.

- `channel_exclude`: A list of channels to filter out for the Batch Effect QC Metrics
- `channel_include`: A list of channels to *only* include for the Batch Effect QC Metrics

The following channels will always be excluded from the TMA Metrics ranking below:
- Au
- Fe
- Na
- Ta
- Noodle


In [ ]:
channel_exclude = ["chan_39", "Biotin", "PDL1", "chan_45"]
channel_include = None

### 2. Compute the QC Longitudinal Control metrics for the set of Control Sample FOVs.

In [ ]:
# Initialize the QC Control Metrics class
qc_control = qc_comp.QCControlMetrics(
    qc_metrics=qc_metrics,
    cohort_path=cohort_path,
    metrics_dir=qc_control_effect_metrics_dir,
)


# Compute the QC metrics for the FOVs provided
qc_control.compute_control_qc_metrics(
    control_sample_name=control_sample_name,
    fovs=fovs,
    channel_exclude=channel_exclude,
    channel_include=channel_include,
)

### 4. Longitudinal Control Heatmap

The following plot below is a heatmap for each Control Sample FOV associated with a particular tissue, in this case it is `"MY_CONTROL_SAMPLE"`.

The QC metrics used to generate this heatmap are normalized by dividing each value by the average of each row, then taking the `log2` of it. Therefore, a value of 1 would be interpreted as be 2 times greater than the row average, and a value of -1 would be 2 times less than the row average.

These plots get saved in a `figures` subfolder within `qc_control_effect_metrics_dir`.

<div align="center">
    <img src="img/nb6_ex_batch_effect_heatmap.png" />
</div>


In [ ]:
qc_metrics_plots.longitudinal_control_heatmap(
    qc_control=qc_control, control_sample_name=control_sample_name, save_figure=True, dpi=300
)